In [1]:
# 載入需要的套件
import os
import numpy as np 
import pandas as pd
import datetime 
import statistics

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data_path = '../data/'
dftrain_2 = pd.read_csv(data_path + 'dftrain_2.csv')
dftrain_2.head()

,Unnamed: 0,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Date_received_weekday,weekday_1,...,weekday_7,Discount_rate_new,Discount_A,Discount_B,Discount_type,Coupon,Date_received_format,Coupon_count,Coupon_Use_count,Target
0,0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,0,...,0,1.00,0,0,0,0,NaN,2,0,-1
1,1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,3,0,...,0,0.95,20,1,1,0,2016-02-17,2,0,0
2,2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,6,0,...,0,0.95,20,1,1,0,2016-03-19,2,0,0
3,3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,5,0,...,0,0.90,200,20,1,0,2016-04-29,1,0,0
4,4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,5,0,...,0,0.90,200,20,1,0,2016-01-29,1,0,0


In [3]:
dftrain_2.shape

(1160742, 25)

In [4]:
dftrain_2=dftrain_2.drop(['Unnamed: 0'], axis=1)
dftrain_2.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Date_received_weekday,weekday_1,weekday_2,...,weekday_7,Discount_rate_new,Discount_A,Discount_B,Discount_type,Coupon,Date_received_format,Coupon_count,Coupon_Use_count,Target
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,0,0,...,0,1.00,0,0,0,0,NaN,2,0,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,3,0,0,...,0,0.95,20,1,1,0,2016-02-17,2,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,6,0,0,...,0,0.95,20,1,1,0,2016-03-19,2,0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,5,0,0,...,0,0.90,200,20,1,0,2016-04-29,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,5,0,0,...,0,0.90,200,20,1,0,2016-01-29,1,0,0


# 建立模型

In [5]:
from sklearn.model_selection import train_test_split, KFold

dftrain_3=dftrain_2
#target不包含-1的群組(沒有領到coupon)
dftrain_3 = dftrain_3[dftrain_3['Target'] != -1].copy()
dftrain_3.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Date_received_weekday,weekday_1,weekday_2,...,weekday_7,Discount_rate_new,Discount_A,Discount_B,Discount_type,Coupon,Date_received_format,Coupon_count,Coupon_Use_count,Target
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,3,0,0,...,0,0.95,20,1,1,0,2016-02-17,2,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,6,0,0,...,0,0.95,20,1,1,0,2016-03-19,2,0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,5,0,0,...,0,0.90,200,20,1,0,2016-04-29,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,5,0,0,...,0,0.90,200,20,1,0,2016-01-29,1,0,0
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,5,0,0,...,0,0.50,10,5,1,0,2016-01-29,1,0,0


In [6]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [7]:
## 選取要建模的指標
original_feature = ['Distance','Discount_rate_new','Discount_A','Discount_B','Discount_type',
                    'Coupon_count','Coupon_Use_count'] + weekdaycols
print("指標數量:",len(original_feature))
print("指標:",original_feature)

指標數量: 14
指標: ['Distance', 'Discount_rate_new', 'Discount_A', 'Discount_B', 'Discount_type', 'Coupon_count', 'Coupon_Use_count', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [8]:
predictors=original_feature

## 資料切割 5:5

In [50]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(dftrain_3, dftrain_3.Target, train_size = 0.5,random_state=50)

print("Train size: {}, #target=1: {}".format(len(x_train), y_train.sum()))
print("Test size: {}, #target=1: {}".format(len(x_test), y_test.sum()))

# 建立模型
regr = linear_model.LogisticRegression()

# 訓練模型
regr.fit(x_train[predictors], y_train)

# 預測測試集
y_pred = regr.predict(x_test[predictors])

C:\Users\Rosa\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train size: 373484, #target=1: 18049
Test size: 373485, #target=1: 18255


C:\Users\Rosa\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [51]:
#使用dataset印出前四筆的預測值
print(regr.predict(x_train[predictors][:4]))

#印出實際的值
print(y_train[:4])

#使用dataset印出前四筆的預測值
print(regr.predict(x_test[predictors][:4]))

#印出實際的值
print(y_test[:4])

[0 0 0 0]
821682     0
603653     1
1071765    0
851993     0
Name: Target, dtype: int64
[0 0 0 0]
187779    0
674097    0
272082    0
240026    0
Name: Target, dtype: int64


In [52]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.9620680884105118


In [53]:
auc_score = roc_auc_score(y_test, y_pred)
print(auc_score)

0.6643494738592292


## 資料切割 7:3

In [54]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(dftrain_3, dftrain_3.Target, train_size = 0.7,random_state=42)

print("Train size: {}, #target=1: {}".format(len(x_train), y_train.sum()))
print("Test size: {}, #target=1: {}".format(len(x_test), y_test.sum()))

# 建立模型
regr = linear_model.LogisticRegression()

# 訓練模型
regr.fit(x_train[predictors], y_train)

# 預測測試集
y_pred = regr.predict(x_test[predictors])

C:\Users\Rosa\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train size: 522878, #target=1: 25448
Test size: 224091, #target=1: 10856


C:\Users\Rosa\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [55]:
#使用dataset印出前四筆的預測值
print(regr.predict(x_train[predictors][:4]))

#印出實際的值
print(y_train[:4])

#使用dataset印出前四筆的預測值
print(regr.predict(x_test[predictors][:4]))

#印出實際的值
print(y_test[:4])

[0 0 0 0]
347546    0
233707    0
320139    0
74593     0
Name: Target, dtype: int64
[0 0 0 0]
1057188    0
769921     0
269190     0
445090     0
Name: Target, dtype: int64


In [56]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.9627071145204403


In [57]:
auc_score = roc_auc_score(y_test, y_pred)
print(auc_score)

0.671574384254252


## 資料切割 1:9

In [58]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(dftrain_3, dftrain_3.Target, train_size = 0.1,random_state=42)

print("Train size: {}, #target=1: {}".format(len(x_train), y_train.sum()))
print("Test size: {}, #target=1: {}".format(len(x_test), y_test.sum()))

# 建立模型
regr = linear_model.LogisticRegression()

# 訓練模型
regr.fit(x_train[predictors], y_train)

# 預測測試集
y_pred = regr.predict(x_test[predictors])

C:\Users\Rosa\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train size: 74696, #target=1: 3606
Test size: 672273, #target=1: 32698


C:\Users\Rosa\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [59]:
#使用dataset印出前四筆的預測值
print(regr.predict(x_train[predictors][:4]))

#印出實際的值
print(y_train[:4])

#使用dataset印出前四筆的預測值
print(regr.predict(x_test[predictors][:4]))

#印出實際的值
print(y_test[:4])

[0 0 1 0]
343746     0
826139     0
622102     0
1082537    0
Name: Target, dtype: int64
[0 0 0 0]
1057188    0
769921     0
269190     0
445090     0
Name: Target, dtype: int64


In [60]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.9619321317381481


In [61]:
auc_score = roc_auc_score(y_test, y_pred)
print(auc_score)

0.6607798722691358
